In [2]:
!pip install transformers requests beautifulsoup4

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from bs4 import BeautifulSoup
import re

BeautifulSoup is a Python library used for parsing and extracting data from HTML and XML documents. It's commonly used in web scraping to navigate and extract information from web pages.

## Models

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

**Note**: there are a number of NLP pretrained models on hugging face (translation, q&a, classification and generation)

## Test

In [127]:
test_tokens = tokenizer.encode("I\'m not a fan of this",return_tensors="pt")

In [128]:
tokenizer.decode(test_tokens[0])

"[CLS] i'm not a fan of this [SEP]"

In [129]:
results = model(test_tokens)

In [130]:
int(torch.argmax(results.logits))+1

2

## Collect reviews

In [53]:
import requests

In [54]:
r = requests.get('https://www.imdb.com/title/tt5090568/reviews')

In [57]:
soup = BeautifulSoup(r.text,"html.parser")

In [84]:
import re
regex = re.compile(r'.*lister-item-content.*')


In [89]:
items = soup.find_all('div',{'class':regex})

## Get data from html

In [146]:
import numpy as np
scores = []
for id,item in enumerate(items):
    ok = False
    for span in item.find_all('span'):
        if span.find_next_sibling('span', class_='point-scale') and span.find_next_sibling('span').text == '/10':
            scores.append(span.text)
            ok = True
            break
    if not ok:
      print(id)
      scores.append(np.nan)


24


In [105]:
comments = []
for id,item in enumerate(items):
    ok = False
    for div in item.find_all('div',class_=re.compile(r'.*text show-more__control.*')):
            comments.append(div.text)
            ok = True
            break
    if not ok: print(id)

In [143]:
titles = []
for id,item in enumerate(items):
    ok = False
    for div in item.find_all('a',class_=re.compile(r'.*title.*')):
            titles.append(div.text)
            ok = True
            break
    if not ok: print(id)

In [138]:
import pandas as pd
import numpy as np

In [148]:
len(comments),len(titles),len(scores)

(25, 25, 25)

## Apply bert

In [150]:
# df = pd.DataFrame([np.array(comments),np.array(titles),np.array(scores)],columns=["comments","titles","scores"])
df = pd.DataFrame({
    "comments": np.array(comments),
    "titles": np.array(titles),
    "scores": np.array(scores)
})

In [153]:
def sentiment_score(comment:str)->int:
  tokens = tokenizer.encode(comment,return_tensors="pt")
  result = model(tokens)
  return int(torch.argmax(result.logits))+1


In [156]:
len(df.comments[0])

687

note that the max input length bert family can get is 512.

In [157]:
df['sentiments'] = df["comments"].apply(lambda x:sentiment_score(x[:512]))

In [159]:
df["title_sents"] = df["titles"].apply(lambda x:sentiment_score(x[:512]))

In [160]:
df

,comments,titles,scores,sentiments,title_sents
0,"Most of this movie is boring, even when there ...",Saved by the final act\n,7,3,5
1,Transformers: Rise of the Beasts (2023) is a m...,It's time to move on from the humans needing ...,6,4,4
2,"""Transformers: Rise of the Beasts"" is the new ...",Forgettable film!\n,5,2,1
3,"As a neutral, I went in there with no real exp...",Better than expected !\n,7,4,5
4,The Transformers movies have always been more ...,Not horrible but also not memorable\n,6,4,3
5,"I usually don't write movie reviews. Since, I ...","If not for showing diversity, the movie did w...",7,2,3
6,Directed by Steven Caple Jr. A runtime of two ...,"""TRANSFORMERS: RISE OF THE BEASTS"" REVIEW\n",7,3,4
7,After the success of 'Bumblebee' I was hoping ...,Quite disappointing\n,4,2,2
8,6.5/10\nWell at least it's better than the Bum...,Transformers: Rise Of The Beasts Is Not Terri...,6,4,3
9,After being disappointed with Age of Extinctio...,They fixed it!!!!!!!!!!\n,8,2,1
